<a href="https://colab.research.google.com/github/mleyvaz/criptografia/blob/main/Cryptography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cryptography

     |████████████████████████████████| 3.2MB 9.3MB/s 


#Importar la librería

In [2]:
from cryptography.fernet import Fernet

Generar la clave

In [15]:
# Generating the key and writing it to a file
def genwrite_key():
    key = Fernet.generate_key()
    with open("pass.key", "wb") as key_file:
        key_file.write(key)
genwrite_key()
%ls

pass.key  sample_data/


# Cargar Clave

In [16]:
def call_key():
  return open("pass.key", "rb").read()

# Encriptar datos

In [17]:
key = call_key()
slogan = "Hello!! Welcome to AIM!!".encode()
a = Fernet(key)
coded_slogan = a.encrypt(slogan)
print(coded_slogan)

b'gAAAAABgJbaikA9jjRHl6uLDixfn46ZoVTWuDjFgUdyGAH_bmZgJBXc9ZG_U3R8N9J7letoTNSyz2RAVJoiloZoSam17AuIoaRc_lJD5cqrGxRKABkJbXR0='


# Desencriptar Datos

In [18]:
key = call_key()
b = Fernet(key)
decoded_slogan = b.decrypt(coded_slogan)
print(decoded_slogan)


b'Hello!! Welcome to AIM!!'


# Asimétrica

In [19]:
import cryptography
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

Generar la clave

In [20]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()

Guardar Clave

In [22]:
# private key
serial_private = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)
with open('private_noshare.pem', 'wb') as f: f.write(serial_private)
    
# public key
serial_pub = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)
with open('public_shared.pem', 'wb') as f: f.write(serial_pub)
%ls

pass.key  private_noshare.pem  public_shared.pem  sample_data/


Leer claves

In [25]:
# make sure the following are imported
# from cryptography.hazmat.backends import default_backend
# from cryptography.hazmat.primitives import serialization
#########      Private device only    ##########
def read_private (filename = "private_noshare.pem"):
    with open(filename, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )
    return private_key
                  
######### Public (shared) device only ##########
def read_public (filename = "public_shared.pem"):
    with open("public_shared.pem", "rb") as key_file:
        public_key = serialization.load_pem_public_key(
            key_file.read(),
            backend=default_backend()
        )
    return public_key

Encriptar

In [48]:
# make sure the following are imported
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
######### Public (shared) device only #########
data = [b'My secret weight', b'My secret id']
public_key = read_public()
open('test.txt', "wb").close() # clear file
for encode in data:
    
    encrypted = public_key.encrypt(
        encode,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    with open('test.txt', "ab") as f: f.write(encrypted)
    %ls

b'My secret weight'
pass.key  private_noshare.pem  public_shared.pem  sample_data/  test.txt
b'My secret id'
pass.key  private_noshare.pem  public_shared.pem  sample_data/  test.txt
